MNIST Dataset Support Vector Machine (SVM) by Micah Borghese

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

ds, info = tfds.load('mnist', split=None, with_info=True)

C:\Users\micah\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Now that the dataset is loaded in, we should probably normalize the vector images (make the pixels either a 0 or a 1). This would probably help with the model's performance. 

However, I'm going to skip that for now and implement it later. It would also be cool to see how an unnormalized vs normalized dataset would compare in classification accuracy.

In [2]:
training_image_arrays = np.array(list(ds['train'].map(lambda x: x['image']).as_numpy_iterator())) # Convert images to NumPy arrays
training_flatten_images = training_image_arrays.reshape(training_image_arrays.shape[0], -1) # Flatten images (28x28) to 1D vectors of 784 pixels
column_names = [f"pixel_{i}" for i in range(training_flatten_images.shape[1])] # Create a pandas DataFrame
X_train = pd.DataFrame(training_flatten_images, columns=column_names) # x is the independent variable (input) of the model, the vector of features (28x28=784 pixels)
y_train = tfds.as_dataframe(ds['train'].map(lambda x: x['label'])) # y is the dependent variable (output) of the model, the label (0-9)

testing_image_arrays = np.array(list(ds['test'].map(lambda x: x['image']).as_numpy_iterator()))
testing_flatten_images = testing_image_arrays.reshape(testing_image_arrays.shape[0], -1)
column_names = [f"pixel_{i}" for i in range(testing_flatten_images.shape[1])]
X_test = pd.DataFrame(testing_flatten_images, columns=column_names)
y_test = tfds.as_dataframe(ds['test'].map(lambda x: x['label']))

In [3]:
from sklearn.svm import SVC
#svm = SVC(kernal='rbf', random_state=0)
#svm.fit(X_train, y_train)

from svm import SVM

svm = SVM()
svm.fit(X_train, y_train)

InvalidIndexError: (0, slice(None, None, None))

After building the SVM, I'm going to save the object in a file using Pickle

In [ ]:
import pickle
pickle.dump(svm, open('svm_model.pkl', 'wb'))

In [ ]:
y_pred = svm.predict(X_test)

AttributeError: 'SVM' object has no attribute 'w'

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 973    0    1    0    0    2    1    1    2    0]
 [   0 1126    3    1    0    1    1    1    2    0]
 [   6    1 1006    2    1    0    2    7    6    1]
 [   0    0    2  995    0    2    0    5    5    1]
 [   0    0    5    0  961    0    3    0    2   11]
 [   2    0    0    9    0  871    4    1    4    1]
 [   6    2    0    0    2    3  944    0    1    0]
 [   0    6   11    1    1    0    0  996    2   11]
 [   3    0    2    6    3    2    2    3  950    3]
 [   3    4    1    7   10    2    1    7    4  970]]


0.9792

The provided confusion matrix offers a detailed snapshot of a classification model's performance across ten classes (or in this case, ten values of pixels since I did not normalize them). Notably, the diagonal elements indicate the number of correct predictions for each class. The model achieved high precision in predicting class 1 (with 1126 correct predictions) and class 2 (with 1006 correct predictions). However, off-diagonal elements are areas of misclassification, such as instances where class 1 was mistakenly predicted as class 2 three times. Nonetheless, he matrix illustrates the model's exceptional performance.

Below the confusion matrix is the accuracy score. My model was able to classify grayscale digits in the MNIST dataset at 97.92% accuracy!